# H1: Velocity + Временные признаки

**H1 (Product):** Скорость операций и cross-border транзакции повышают риск → шаг-up аутентификация при превышении порогов.

In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features, customer_velocity
from src.validation import split_time_aware, bootstrap_pr_auc
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df = load_transactions(TX); fx = load_fx(FX)
df = convert_to_usd(df, fx); df = unpack_last_hour_activity(df); df = add_basic_time_features(df)

# Baseline
train, test = split_time_aware(df)
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])
pipe_base = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
p_base = pipe_base.predict_proba(X_te)[:,1]

# Candidate: +velocity
df_v = customer_velocity(df)
train_v, test_v = split_time_aware(df_v)
y_tr_v, y_te_v = train_v['is_fraud'].astype(int), test_v['is_fraud'].astype(int)
X_tr_v, X_te_v = train_v.drop(columns=['is_fraud']), test_v.drop(columns=['is_fraud'])
pipe_v = Pipeline([('prep', build_preprocessor(X_tr_v)), ('clf', build_logreg())]).fit(X_tr_v, y_tr_v)
p_v = pipe_v.predict_proba(X_te_v)[:,1]

base_metrics = eval_pack(y_te, p_base); cand_metrics = eval_pack(y_te_v, p_v)
delta, lo, hi = bootstrap_pr_auc(y_te_v.to_numpy(), p_base, p_v, B=500)
base_metrics, cand_metrics, {'delta_pr':delta, '95%CI':(lo,hi)}

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

({'roc_auc': 0.5001122214983779,
  'pr_auc': 0.1980998640895631,
  'threshold_at_precision': 0.9,
  'thr_value': None,
  'recall_at_precision': None,
  'precision_achieved': None},
 {'roc_auc': 0.5001122214983779,
  'pr_auc': 0.1980998640895631,
  'threshold_at_precision': 0.9,
  'thr_value': None,
  'recall_at_precision': None,
  'precision_achieved': None},
 {'delta_pr': 0.0, '95%CI': (0.0, 0.0)})

### Краткий вывод по H1 (Velocity + время):
* Baseline и кандидат с velocity/time показывают одинаковые метрики: ROC-AUC ≈ 0.5001, PR-AUC ≈ 0.1981 (≈ доля позитивов), ∆PR-AUC = 0.0 с 95% CI = (0.0; 0.0).
* Целевой Precision ≥ 0.90 недостижим на текущем скоупе признаков (порог не найден).

### Интерпретация

Модель находится на уровне случайного угадывания; добавление velocity/времени не дало сигнала. Вероятные причины:
1.	Потеря информативности из-за препроцессинга: жесткий порог min_frequency=50 в OHE мог выкинуть большинство категорий; числовые признаки могли стать почти константными после импутации.
2.	Пустые/некорректные velocity-окна: если временной индекс/гранулярность не позволяют посчитать rolling, новые фичи заполняются нулями/NaN и не помогают.
3.	Неверная конвертация сумм (массовые NaN после join с FX по дате → затем медианная импутация превращает amount_usd в слабый сигнал).

### Что сделать дальше (минимальные правки)
* Снизить min_frequency в OneHotEncoder до 10 или убрать, чтобы не терять категориальные признаки.
* Провести sanity-чек фичей после препроцессинга/импутации: доля NaN, доля константных/квази-константных колонок, дисперсии.
* Проверить конвертацию валют: долю NaN в rate_to_currency после join и покрытие дат (при необходимости — ffill по дате).
* Убедиться, что customer_velocity реально создаёт вариацию (распределения cust_tx_count_{1,6,24}h, time_since_prev_s ≠ константа).
* Как бенчмарк: запустить дерево/LightGBM (умеют работать с пропусками) — если качество растёт, проблема именно в признаках/импутации для логистической регрессии.

Итог по H1 на текущем срезе: гипотеза не подтверждена; необходима донастройка препроцессинга и валидация корректности источников сигнала (velocity, суммы, категории).